In [ ]:
import os as os
import numpy as numpy
# import scipy as scipy
from scipy import ndimage
from six.moves import cPickle as pickle
import random as random
import itertools as itertools
import tensorflow as tf
import matplotlib.pyplot as plt
import math as math

In [ ]:
basePath = "./lfw/"
numFaces = 50

In [ ]:
def readSingleImage(fileName):
    return(ndimage.imread(fileName, mode="RGB"))

   
def loadPicturesFromFiles(basePath):
    allNames = os.listdir(basePath)[1:numFaces]
    allPictures = {}
    for personName in allNames:
        personPictureDirectory = os.path.join(basePath, personName)
        if (not (personName[0] == ".")) & os.path.isdir(personPictureDirectory):
            print("Reading faces of " + personName + "...", end="")
            pictureFiles = os.listdir(personPictureDirectory)
            pictureFiles = list(map(os.path.join, [personPictureDirectory] * len(pictureFiles), pictureFiles))
            pictures = list(map(readSingleImage, pictureFiles))
            print(" DONE (" + str(len(pictures)) + " read)")
            allPictures[personName] = pictures
    return(allPictures)

allImagesDict = loadPicturesFromFiles("./lfw")
pickle.dump(allImagesDict, open("./autoencoder images.pickle", "wb"))

In [ ]:
#allImages = pickle.load(open("./autoencoder images.pickle", "wb"))

In [ ]:
def splitImagesToTrainingAndTestingSets(allImagesDict, trainingPortion = 0.75):
    trainingImages = []
    trainingLabels = []
    testingImages  = []
    testingLabels  = []
    for personNames, pictures in allImagesDict.items():
        if (random.uniform(0,1) < trainingPortion):
            trainingLabels.append([personNames] * len(pictures))
            trainingImages.append(pictures)
        else:
            testingLabels.append([personNames] * len(pictures))
            testingImages.append(pictures)
    return((
        list(itertools.chain(*trainingImages)),
        list(itertools.chain(*trainingLabels)),
        list(itertools.chain(*testingImages)),
        list(itertools.chain(*testingLabels))))

[originalTrainingImages, trainingLabels, originalTestingImages, testingLabels] = splitImagesToTrainingAndTestingSets(allImagesDict)

In [ ]:
def expandSingleImage(img):
    expandedImage = numpy.zeros((256, 256, 3), dtype=numpy.uint8)
    expandedImage[:img.shape[0], :img.shape[1], :] = img
    return(expandedImage)

def expandAllImages(trainingImages, testingImages):
    trainingImages = list(map(expandSingleImage, trainingImages))
    testingImages  = list(map(expandSingleImage, testingImages))
    return((trainingImages, testingImages))

[expandedTrainingImages, expandedTestingImages] = expandAllImages(originalTrainingImages, originalTestingImages)

# Show Loaded Images

### Training Images

In [ ]:
numImageRows = 1 + (len(expandedTrainingImages) // 10)
plt.figure(figsize=(50, 5 * numImageRows))
for imgID in range(len(expandedTrainingImages)):
    plt.subplot(numImageRows, 10, imgID + 1)
    plt.imshow(expandedTrainingImages[imgID])
plt.show()

### Testing Images

In [ ]:
numImageRows = 1 + (len(expandedTestingImages) // 10)
plt.figure(figsize=(50, numImageRows * 5))
for imgID in range(len(expandedTestingImages)):
    plt.subplot(numImageRows, 10, imgID + 1)
    plt.imshow(expandedTestingImages[imgID])
plt.show()

### Scale Pixel Values in Training/Testing Images to 0...1 scale + function to back-scale to 0...255 range

In [ ]:
def zeroOneScaleColoursInImages(trainingImages, testingImages):
    trainingImages = list(map(lambda image: image / 255.0, trainingImages))
    testingImages  = list(map(lambda image: image / 255.0, testingImages))
    return((trainingImages, testingImages))

def backScaleColoursInImages(imageList):
    imageList = list(map(lambda image: image * 255, imageList))
    imageList = list(map(lambda image: image.astype(numpy.uint8), imageList))
    return(imageList)

[trainingImages, testingImages] = zeroOneScaleColoursInImages(expandedTrainingImages, expandedTestingImages)

# Simple Autoencoder

### Define Network and Training & Loss Function

In [ ]:
tf.reset_default_graph()
inputLayer    = tf.placeholder(tf.float32, (None, 256, 256, 3))
encodingLayer = tf.contrib.layers.conv2d(inputs = inputLayer, num_outputs = 12, kernel_size = 3, stride = 2, padding = "SAME")
outputLayer   = tf.contrib.layers.conv2d_transpose(inputs = encodingLayer, num_outputs = 3, kernel_size = 3, stride = 2, padding = "SAME")
print(inputLayer.shape)
print(encodingLayer.shape)
print(outputLayer.shape)

In [ ]:
# calculate the loss and optimize the network
lossFunction = tf.reduce_mean(tf.square(outputLayer - inputLayer))  # calaculate the mean square error loss
optimisation = tf.train.AdamOptimizer().minimize(lossFunction)

# initialize the network
init = tf.global_variables_initializer()

### Training of the Network

In [ ]:
def showProgressOnTestingImages(inputTstImgs, codedTstImgs, outputTstImgs):
    inputTstImgs  = backScaleColoursInImages(inputTstImgs)
    codedTstImgs  = backScaleColoursInImages(codedTstImgs)
    outputTstImgs = backScaleColoursInImages(outputTstImgs)
        
    plt.figure(figsize=(7.5, 2.5 * len(inputTstImgs)))
    for imgID in range(len(inputTstImgs)):
        plt.subplot(len(inputTstImgs)+1, 3, imgID * 3 + 1)
        plt.imshow(inputTstImgs[imgID])

        plt.subplot(len(inputTstImgs)+1, 3, imgID * 3 + 2)
        plt.imshow(codedTstImgs[imgID])

        plt.subplot(len(inputTstImgs)+1, 3, imgID * 3 + 3)
        plt.imshow(outputTstImgs[imgID])
    plt.show()



In [ ]:
session = tf.Session()
session.run(init)

for epochID in range(100):
    epochTrainingImages = random.sample(trainingImages, len(trainingImages))
    batchLen = len(epochTrainingImages) // 10
    print("Running epoch {}".format(epochID), end="")
    for batchID in range(batchLen):
        batchImages = epochTrainingImages[batchID * batchLen : min((batchID+1) * batchLen, len(epochTrainingImages))]
        if len(batchImages) == 0:
            continue
        _, trainLoss = session.run([optimisation, lossFunction], feed_dict = {inputLayer: batchImages})
        print(".", end="")
    testLoss = session.run([lossFunction], feed_dict = {inputLayer: testingImages})
    print(" loss on testing was {}".format(testLoss))
    if (epochID % 50 == 0):
        codedTstImgs, outputTstImgs = session.run([encodingLayer, outputLayer], feed_dict={inputLayer: testingImages})
        showProgressOnTestingImages(testingImages, codedTstImgs, outputTstImgs)
        

# Compressing AutoEncoder

### Define Network and Loss Function

In [ ]:
tf.reset_default_graph()
inputLayer = tf.placeholder(tf.float32, (None, 256, 256, 3))
covLayer1 = tf.contrib.layers.conv2d(inputs = inputLayer,
                                     num_outputs = 64,
                                     kernel_size = 5,
                                     stride = 2,
                                     padding = "SAME")
covLayer2 = tf.contrib.layers.conv2d(inputs = covLayer1,
                                     num_outputs = 32,
                                     kernel_size = 5,
                                     stride = 2,
                                     padding = "SAME")
codeLayer = tf.contrib.layers.conv2d(inputs = covLayer2,
                                     num_outputs = 16,
                                     kernel_size = 5,
                                     stride = 2,
                                     padding = "SAME")
decovLayer2 = tf.contrib.layers.conv2d_transpose(inputs = codeLayer,
                                     num_outputs = 32,
                                     kernel_size = 5,
                                     stride = 2,
                                     padding = "SAME")
decovLayer1 = tf.contrib.layers.conv2d_transpose(inputs = decovLayer2,
                                     num_outputs = 64,
                                     kernel_size = 5,
                                     stride = 2,
                                     padding = "SAME")
outputLayer = tf.contrib.layers.conv2d_transpose(inputs = decovLayer1,
                                               num_outputs = 3,
                                               kernel_size = 5,
                                               stride = 2, 
                                               padding = "SAME")

#Print out layer shapes - for validation
print(inputLayer.shape)
print(covLayer1.shape)
print(covLayer2.shape)
print(codeLayer.shape)
print(decovLayer2.shape)
print(decovLayer1.shape)
print(outputLayer.shape)

In [ ]:
# calculate the loss and optimize the network
lossFunction = tf.reduce_mean(tf.square(outputLayer - inputLayer))  # calaculate the mean square error loss
optimisation = tf.train.AdamOptimizer(learning_rate=0.001).minimize(lossFunction)

# initialize the network
init = tf.global_variables_initializer()

In [ ]:
session = tf.Session()
session.run(init)

for epochID in range(100):
    epochTrainingImages = random.sample(trainingImages, len(trainingImages))
    batchLen = len(epochTrainingImages) // 10
    print("Running epoch {}".format(epochID), end="")
    for batchID in range(batchLen):
        batchImages = epochTrainingImages[batchID * batchLen : min((batchID+1) * batchLen, len(epochTrainingImages))]
        if len(batchImages) == 0:
            continue
        _, trainLoss = session.run([optimisation, lossFunction], feed_dict = {inputLayer: batchImages})
        print(".", end="")
    testLoss = session.run([lossFunction], feed_dict = {inputLayer: testingImages})
    print(" loss on testing was {}".format(testLoss))

In [ ]:
outputTstImgs = session.run([outputLayer], feed_dict={inputLayer: testingImages})[0]
outputTstImgs = backScaleColoursInImages(outputTstImgs)
inputTstImgs = backScaleColoursInImages(testingImages)

plt.figure(figsize=(5, 2.5 * len(inputTstImgs)))
for imgID in range(len(outputTstImgs)):
    plt.subplot(len(outputTstImgs)+1, 2, imgID * 2 + 1)
    plt.imshow(inputTstImgs[imgID])

    plt.subplot(len(outputTstImgs)+1, 2, imgID * 2 + 2)
    plt.imshow(outputTstImgs[imgID])
plt.show()

In [ ]:
imgCode = session.run(codeLayer, feed_dict = {inputLayer: testingImages[0]})

In [ ]:
tf.

# Denoising AutoEncoder

### Add Noise to Training and Testing Images -- Randomly Blacken colour channel in pixels

In [ ]:
def addRandomNoiseToSingleImage(img, noiseLevel):
    imageDimensions = img.shape
    noise = numpy.random.rand(imageDimensions[0], imageDimensions[1], imageDimensions[2])
    noiseIter = numpy.nditer([noise, None])
    for i, out in noiseIter:
        if i > noiseLevel:
            out[...] = 1
        else:
            out[...] = 0
    imageMask = noiseIter.operands[1]
    return(img * imageMask)

def addRandomNoiseToAllImages(allImages, noiseLevel):
    return(list(map(lambda img : addRandomNoiseToSingleImage(img, noiseLevel), allImages)))

noisedTrainingImages = addRandomNoiseToAllImages(trainingImages, 0.05)
noisedTestingImages = addRandomNoiseToAllImages(testingImages, 0.05)
           
numImages = len(trainingImages[:5])
plt.figure(figsize=(5, 2.5 * numImages))
for imgID in range(numImages):
    plt.subplot(numImages+1, 2, imgID * 2 + 1)
    plt.imshow(trainingImages[imgID])

    plt.subplot(numImages+1, 2, imgID * 2 + 2)
    plt.imshow(noisedTrainingImages[imgID])
plt.show()

numImages = len(testingImages[:5])
plt.figure(figsize=(5, 2.5 * numImages))
for imgID in range(numImages):
    plt.subplot(numImages+1, 2, imgID * 2 + 1)
    plt.imshow(testingImages[imgID])

    plt.subplot(numImages+1, 2, imgID * 2 + 2)
    plt.imshow(noisedTestingImages[imgID])
plt.show()

In [ ]:
def randomShuffleImages(inputImages, noisedInputImages):
    idx = (random.sample(range(len(inputImages)), len(inputImages)))
    inputImages = list(map(inputImages.__getitem__, idx))
    noisedInputImages = list(map(noisedInputImages.__getitem__, idx))
    return(inputImages, noisedInputImages)

### Define Network and Loss Function

In [ ]:
tf.reset_default_graph()
noisedInputLayer = tf.placeholder(tf.float32, (None, 256, 256, 3))
decovLayer1 = tf.contrib.layers.conv2d_transpose(inputs = noisedInputLayer,
                                     num_outputs = 32,
                                     kernel_size = 5,
                                     stride = 2,
                                     padding = "SAME")
# codeLayer = tf.contrib.layers.conv2d_transpose(inputs = decovLayer1,
#                                                num_outputs = 16,
#                                                kernel_size = 5,
#                                                stride = 2, 
#                                                padding = "SAME")
# covLayer1 = tf.contrib.layers.conv2d(inputs = codeLayer,
#                                      num_outputs = 32,
#                                      kernel_size = 5,
#                                      stride = 2,
#                                      padding = "SAME")
outputLayer = tf.contrib.layers.conv2d(inputs = decovLayer1,
                                     num_outputs = 3,
                                     kernel_size = 5,
                                     stride = 2,
                                     padding = "SAME")

#Print out layer shapes - for validation
print(inputLayer.shape)
print(decovLayer1.shape)
print(codeLayer.shape)
print(covLayer1.shape)
print(outputLayer.shape)

In [ ]:
# calculate the loss and optimize the network
originalInputImage = tf.placeholder(tf.float32, (None, 256, 256, 3))
lossFunction = tf.reduce_mean(tf.square(outputLayer - originalInputImage))  # calaculate the mean square error loss
optimisation = tf.train.AdamOptimizer(learning_rate=0.001).minimize(lossFunction)

# initialize the network
init = tf.global_variables_initializer()

In [ ]:
session = tf.Session()
session.run(init)

for epochID in range(100):
    epochTrainingImages, epochNoisedTrainingImages = randomShuffleImages(trainingImages, noisedTrainingImages)
    
    batchLen = len(epochTrainingImages) // 10
    print("Running epoch {}".format(epochID), end="")
    for batchID in range(batchLen):
        batchImages       = epochTrainingImages[batchID * batchLen : min((batchID+1) * batchLen, len(epochTrainingImages))]
        batchNoisedImages = epochNoisedTrainingImages[batchID * batchLen : min((batchID+1) * batchLen, len(epochNoisedTrainingImages))]
        if len(batchImages) == 0:
            continue
        _, trainLoss = session.run([optimisation, lossFunction], feed_dict = {noisedInputLayer: batchNoisedImages, originalInputImage: batchImages})
        print(".", end="")
    testLoss = session.run([lossFunction], feed_dict = {noisedInputLayer: noisedTestingImages, originalInputImage: testingImages})
    print(" loss on testing was {}".format(testLoss))

In [ ]:
outputTstImgs = session.run([outputLayer], feed_dict={noisedInputLayer: noisedTestingImages, originalInputImage: testingImages})[0]
outputTstImgs = backScaleColoursInImages(outputTstImgs)
inputTstImgs = backScaleColoursInImages(noisedTestingImages)

plt.figure(figsize=(5, 2.5 * len(inputTstImgs)))
for imgID in range(len(outputTstImgs)):
    plt.subplot(len(outputTstImgs)+1, 2, imgID * 2 + 1)
    plt.imshow(inputTstImgs[imgID])

    plt.subplot(len(outputTstImgs)+1, 2, imgID * 2 + 2)
    plt.imshow(outputTstImgs[imgID])
plt.show()

In [ ]:
seq(1,20)

# Backup and tests

In [ ]:
inputLayer = tf.placeholder(tf.float32, (None, 256, 256, 3))

In [ ]:
encLayer1 = tf.contrib.layers.conv2d(inputs = inputLayer, 
                                     num_outputs = 50,
                                     kernel_size = 11,
                                     stride = 4, 
                                     padding = "SAME")

encLayer2 = tf.contrib.layers.conv2d(inputs = encLayer1,
                                     num_outputs = 25,
                                     kernel_size = 7,
                                     stride = 2,
                                     padding = "SAME")

codeLayer = tf.contrib.layers.conv2d(inputs = encLayer2,
                                     num_outputs = 10,
                                     kernel_size = 5,
                                     stride = 2,
                                     padding = "SAME")

In [ ]:
decLayer2 = tf.contrib.layers.conv2d_transpose(inputs = codeLayer,
                                               num_outputs = 25,
                                               kernel_size = 5,
                                               stride = 2,
                                               padding="SAME")

decLayer1 = tf.contrib.layers.conv2d_transpose(inputs = decLayer2,
                                               num_outputs = 50,
                                               kernel_size = 7,
                                               stride = 2, 
                                               padding = "SAME")

outputLayer = tf.contrib.layers.conv2d_transpose(inputs = decLayer1,
                                               num_outputs = 3,
                                               kernel_size = 11,
                                               stride = 4, 
                                               padding = "SAME")

In [ ]:
print(inputLayer.shape)
print(encLayer1.shape)
print(encLayer2.shape)
print(codeLayer.shape)

print(decLayer2.shape)
print(decLayer1.shape)
print(outputLayer.shape)

In [ ]:
# calculate the loss and optimize the network
lossFunction = tf.reduce_mean(tf.square(outputLayer - inputLayer))  # calaculate the mean square error loss
optimisation = tf.train.AdamOptimizer(learning_rate=0.001).minimize(lossFunction)

# initialize the network
init = tf.global_variables_initializer()

In [ ]:
testingImages[0].shape

In [ ]:
session = tf.Session()
session.run(init)

for epochID in range(100):
    _, trainLoss = session.run([optimisation, lossFunction], feed_dict = {inputLayer: trainingImages})
    testLoss = session.run([lossFunction], feed_dict = {inputLayer: testingImages})
    print("In go {} the loss on training was {} and loss on testing was {}".format(epochID, trainLoss, testLoss))

In [ ]:
modeledTrainingImages = session.run([outputLayer], feed_dict={inputLayer: trainingImages})[0]
modeledTrainingImagesOrigColours = backScaleColoursInImages(modeledTrainingImages)
trainingImagesOrigColours = backScaleColoursInImages(trainingImages)

In [ ]:
plt.figure(figsize=(5, 2.5 * len(modeledTrainingImages)))
for imgID in range(len(modeledTrainingImages)):
    plt.subplot(len(modeledTrainingImages)+1, 2, imgID * 2 + 1)
    plt.imshow(trainingImagesOrigColours[imgID])

    plt.subplot(len(modeledTrainingImages)+1, 2, imgID * 2 + 2)
    plt.imshow(modeledTrainingImagesOrigColours[imgID])
plt.show()

In [ ]:
len(modeledTrainingImages[0][0][0][0])

In [ ]:

plt.figure()
plt.imshow(originalColourRangeImages[5])
plt.show()

In [ ]:
def backScaleColoursInImages1(imageList):
    imageList = list(map(lambda image: image * 255, imageList))
    imageList = list(map(lambda image: image.astype(numpy.uint8), imageList))
    return(imageList)
    

img = backScaleColoursInImages1(trainingImages)[0]
plt.figure()
plt.imshow(img)
plt.show()

# AutoEncoder Simple

In [ ]:
tf.reset_default_graph()
inputLayer = tf.placeholder(tf.float32, (None, 256, 256, 3))
codeLayer = tf.contrib.layers.conv2d(inputs = inputLayer,
                                     num_outputs = 20,
                                     kernel_size = 3,
                                     stride = 1,
                                     padding = "SAME")
outputLayer = tf.contrib.layers.conv2d_transpose(inputs = codeLayer,
                                               num_outputs = 3,
                                               kernel_size = 3,
                                               stride = 1, 
                                               padding = "SAME")

# calculate the loss and optimize the network
lossFunction = tf.reduce_mean(tf.square(outputLayer - inputLayer))  # calaculate the mean square error loss
optimisation = tf.train.AdamOptimizer(learning_rate=0.001).minimize(lossFunction)

# initialize the network
init = tf.global_variables_initializer()

#Print out layer shapes - for validation
print(inputLayer.shape)
print(codeLayer.shape)
print(outputLayer.shape)

In [ ]:
session = tf.Session()
session.run(init)

for epochID in range(50):
    epochTrainingImages = random.sample(trainingImages, len(trainingImages))
    batchLen = len(epochTrainingImages) // 10
    print("Running epoch {}".format(epochID), end="")
    for batchID in range(batchLen):
        batchImages = epochTrainingImages[batchID * batchLen : min((batchID+1) * batchLen, len(epochTrainingImages))]
        _, trainLoss = session.run([optimisation, lossFunction], feed_dict = {inputLayer: batchImages})
        print(".", end="")
    testLoss = session.run([lossFunction], feed_dict = {inputLayer: testingImages})
    print(" loss on testing was {}".format(testLoss))

In [ ]:
modeledTestingImages = session.run([outputLayer], feed_dict={inputLayer: testingImages})[0]
modeledTestingImagesOrigColours = backScaleColoursInImages(modeledTestingImages)
testingImagesOrigColours = backScaleColoursInImages(testingImages)

In [ ]:
plt.figure(figsize=(5, 2.5 * len(modeledTestingImagesOrigColours)))
for imgID in range(len(modeledTestingImagesOrigColours)):
    plt.subplot(len(modeledTestingImagesOrigColours)+1, 2, imgID * 2 + 1)
    plt.imshow(testingImagesOrigColours[imgID])

    plt.subplot(len(modeledTestingImagesOrigColours)+1, 2, imgID * 2 + 2)
    plt.imshow(modeledTestingImagesOrigColours[imgID])
plt.show()

# AutoEncoder For Compression

In [ ]:
tf.reset_default_graph()
inputLayer = tf.placeholder(tf.float32, (None, 256, 256, 3))
covLayer1 = tf.contrib.layers.conv2d(inputs = inputLayer,
                                     num_outputs = 64,
                                     kernel_size = 5,
                                     stride = 2,
                                     padding = "SAME")
covLayer2 = tf.contrib.layers.conv2d(inputs = covLayer1,
                                     num_outputs = 32,
                                     kernel_size = 5,
                                     stride = 2,
                                     padding = "SAME")
codeLayer = tf.contrib.layers.conv2d(inputs = covLayer2,
                                     num_outputs = 16,
                                     kernel_size = 5,
                                     stride = 2,
                                     padding = "SAME")
decovLayer2 = tf.contrib.layers.conv2d_transpose(inputs = codeLayer,
                                     num_outputs = 32,
                                     kernel_size = 5,
                                     stride = 2,
                                     padding = "SAME")
decovLayer1 = tf.contrib.layers.conv2d_transpose(inputs = decovLayer2,
                                     num_outputs = 64,
                                     kernel_size = 5,
                                     stride = 2,
                                     padding = "SAME")
outputLayer = tf.contrib.layers.conv2d_transpose(inputs = decovLayer1,
                                               num_outputs = 3,
                                               kernel_size = 5,
                                               stride = 2, 
                                               padding = "SAME")

# calculate the loss and optimize the network
lossFunction = tf.reduce_mean(tf.square(outputLayer - inputLayer))  # calaculate the mean square error loss
optimisation = tf.train.AdamOptimizer(learning_rate=0.001).minimize(lossFunction)

# initialize the network
init = tf.global_variables_initializer()

#Print out layer shapes - for validation
print(inputLayer.shape)
print(covLayer1.shape)
print(covLayer2.shape)
print(codeLayer.shape)
print(decovLayer2.shape)
print(decovLayer1.shape)
print(outputLayer.shape)

In [ ]:
session = tf.Session()
session.run(init)

for epochID in range(500):
    epochTrainingImages = random.sample(trainingImages, len(trainingImages))
    batchLen = len(epochTrainingImages) // 10
    print("Running epoch {}".format(epochID), end="")
    for batchID in range(batchLen):
        batchImages = epochTrainingImages[batchID * batchLen : min((batchID+1) * batchLen, len(epochTrainingImages))]
        if len(batchImages) == 0:
            continue
        _, trainLoss = session.run([optimisation, lossFunction], feed_dict = {inputLayer: batchImages})
        print(".", end="")
    testLoss = session.run([lossFunction], feed_dict = {inputLayer: testingImages})
    print(" loss on testing was {}".format(testLoss))

In [ ]:
list(map(lambda img : img.shape, trainingImages))

In [ ]:
modeledTestingImages = session.run([outputLayer], feed_dict={inputLayer: testingImages})[0]
modeledTestingImagesOrigColours = backScaleColoursInImages(modeledTestingImages)
testingImagesOrigColours = backScaleColoursInImages(testingImages)

In [ ]:
plt.figure(figsize=(5, 2.5 * len(modeledTestingImagesOrigColours)))
for imgID in range(len(modeledTestingImagesOrigColours)):
    plt.subplot(len(modeledTestingImagesOrigColours)+1, 2, imgID * 2 + 1)
    plt.imshow(testingImagesOrigColours[imgID])

    plt.subplot(len(modeledTestingImagesOrigColours)+1, 2, imgID * 2 + 2)
    plt.imshow(modeledTestingImagesOrigColours[imgID])
plt.show()